In [ ]:
import pyodbc
import cv2
import os
from deepface import DeepFace
from datetime import datetime


# Cargar el clasificador de rostros
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Iniciar la captura de video desde la webcam
cap = cv2.VideoCapture(0)

scaling_factor = 1  # Parámetro que especifica cuánto se reduce el tamaño de la imagen en cada iteración

while True:
    # Capturar un frame de video
    ret, frame = cap.read()

    # Redimensionar el frame de video
    frame = cv2.resize(frame, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)

    # Detectar rostros en el frame
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=3)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        
        # Analizar la emoción y el género del rostro
        analysis = DeepFace.analyze(face, actions=['emotion', 'gender'], enforce_detection=False)
        emotion = analysis[0]['dominant_emotion']
        gender = analysis[0]['dominant_gender']
        
        # Agregar texto de emoción y género al frame
        emotion_text = "Emotion: " + emotion
        gender_text = "Gender: " + gender
        cv2.putText(frame, emotion_text, (x, y-40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        cv2.putText(frame, gender_text, (x, y-20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 3)
        
        # Generar la cadena de tiempo
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Convertir la cadena de tiempo a un objeto datetime
        timestamp_datetime = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
        
        # Insertar datos en la base de datos
        cursor = conn.cursor()
        cursor.execute("INSERT INTO EMOTIONS (Emotion, Gender, CreatedAt) VALUES (?, ?, ?)",
                       (emotion, gender, timestamp_datetime))
        conn.commit()

    # Mostrar el frame resultante
    cv2.imshow('frame', frame)
    
    # Presionar 'q' para salir
    if cv2.waitKey(1) == ord('q'):
        break

# Liberar la captura y cerrar todas las ventanas
cap.release()
cv2.destroyAllWindows()
conn.close()